In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.datasets as d_sets
from torch.utils.data import DataLoader as d_loader
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import os
import os.path as osp
import logging
from collections import OrderedDict
import json
from datetime import datetime
import os
import math
import numpy as np

from torchvision.utils import make_grid
import numpy as np
import torch
from torch.utils.data import Dataset

import logging
from re import split


In [2]:
!nvidia-smi

Tue Nov 25 14:12:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090 D      Off |   00000000:01:00.0 Off |                  Off |
| 30%   45C    P0            112W /  425W |    1102MiB /  24564MiB |     29%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch

# 检查可用的 GPU 数量
print("Available GPUs count:", torch.cuda.device_count())

# 打印每个 GPU 的详细信息
for i in range(torch.cuda.device_count()):
    print("GPU", i, ":", torch.cuda.get_device_name(i))

Available GPUs count: 8
GPU 0 : NVIDIA GeForce RTX 4090 D
GPU 1 : NVIDIA GeForce RTX 4090 D
GPU 2 : NVIDIA GeForce RTX 4090 D
GPU 3 : NVIDIA GeForce RTX 4090 D
GPU 4 : NVIDIA GeForce RTX 4090 D
GPU 5 : NVIDIA GeForce RTX 4090 D
GPU 6 : NVIDIA GeForce RTX 4090 D
GPU 7 : NVIDIA GeForce RTX 4090 D


In [4]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

2025-11-25 14:12:21.544341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764079941.560007 1166479 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764079941.564785 1166479 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764079941.581903 1166479 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764079941.581939 1166479 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764079941.581943 1166479 computation_placer.cc:177] computation placer alr

In [5]:
import torch
torch.cuda.set_device(2)

In [6]:
def tensor2numpy(tensor, mean, std, mean_input_wind_lr, std_input_wind_lr):
    '''
    Converts a torch Tensor into a numpy array and de-normalizes it using given mean and std
    Input: 3D(1,H,W) or 4D(1,1,H,W)
    Output: 3D(1,H,W) or 4D(1,1,H,W), float32
    '''
    if tensor.dim() == 3 and tensor.size(0) == 1:
        tensor = tensor.squeeze(0)  # Remove the first dimension (C, H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean and std
        tensor = tensor * std + mean
        
        img_np = tensor.numpy()
        
        
    
    elif tensor.dim() == 4 and tensor.size(0) == 1 and tensor.size(1) == 1:
        tensor = tensor.squeeze(0).squeeze(0)  # Remove the first two dimensions (H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean and std
        tensor = tensor * std + mean
        
        img_np = tensor.numpy()
        
    elif tensor.dim() == 4 and tensor.size(0) == 1 and tensor.size(1) == 2:
        # Extract the third channel
        tensor = tensor.squeeze(0)  # Remove the first dimension (3, H, W)
        tensor = tensor[1, :, :]  # Extract the third channel (H, W)
        tensor = tensor.float().cpu()
        
        # De-normalize using the given mean_input_wind_lr and std_input_wind_lr
        tensor = tensor * std_input_wind_lr + mean_input_wind_lr
        
        img_np = tensor.numpy()
        
       
    
    else:
        raise TypeError('Only support 3D tensor with first dimension as 1 or 4D tensor with first two dimensions as 1. But received with dimension: {:d} and size: {}'.format(tensor.dim(), tensor.size()))
    
    return img_np.astype(np.float32)

def save_img(img, img_path):
    '''
    Save an image as a .npy file.
    '''
    np.save(img_path, img)


def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    max_value = max(np.max(img1), np.max(img2))
    return 20 * math.log10(max_value / math.sqrt(mse))

def calculate_mse(img1, img2):
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    return mse


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset

class LRHR_dataset(Dataset):
    def __init__(self, input_file, output_file):
        """
        初始化数据集类
        :param input_file: 包含输入数据的npy文件路径（combined_data.npy）
        :param output_file: 包含输出数据的npy文件路径（filtered_combined_wind_speed_usa.npy）
        """
        # 加载数据
        self.input_data = np.load(input_file)
        self.output_data = np.load(output_file)

        # 对输入数据进行标准化
        self.normalized_input = np.zeros_like(self.input_data)
        for i in range(self.input_data.shape[1]):  # 对每个变量进行循环
            mean = np.mean(self.input_data[:, i, :, :], axis=0, keepdims=True)
            std = np.std(self.input_data[:, i, :, :], axis=0, keepdims=True)
            if np.any(std == 0):  # 如果标准差中有任何一个为零，则跳过该变量
                print(f"Variable {i} has at least one std=0, skipping normalization for this variable.")
                self.normalized_input[:, i, :, :] = self.input_data[:, i, :, :]
            else:
                self.normalized_input[:, i, :, :] = (self.input_data[:, i, :, :] - mean) / std
            if i == 1:
                self.mean_i2 = mean
                self.std_i2 = std
        # 对输出数据进行标准化
        self.mean_output = np.mean(self.output_data, axis=0, keepdims=True)
        self.std_output = np.std(self.output_data, axis=0, keepdims=True)
        self.normalized_output = (self.output_data - self.mean_output) / self.std_output

    def __len__(self):
        """
        返回数据集中的样本数
        """
        return self.input_data.shape[0]

    def __getitem__(self, index):
        """
        根据索引获取数据集中的样本
        :param index: 数据的索引值
        """
        input_sample = self.normalized_input[index]
        output_sample = self.normalized_output[index]

        # 将NumPy数组转换为torch tensors
        input_tensor = torch.from_numpy(input_sample).float()
        output_tensor = torch.from_numpy(output_sample[np.newaxis, :, :]).float()

        return input_tensor, output_tensor
    
    def get_output_mean_std(self):
        """
        返回输出数据的均值和标准差
        """
        return self.mean_output, self.std_output
    def get_input_i2_mean_std(self):
        """
        返回input_data中i=3的均值和标准差
        """
        return self.mean_i2, self.std_i2
# 使用示例
# dataset = WindSpeedDataset('combined_data.npy', 'filtered_combined_wind_speed_usa.npy')
# input_sample, output_sample = dataset[0]


In [8]:
import torch
from torch import nn

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 2, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        x = self.mlp_head(x)
        x = x.view(b, 1, 64, 64)
        
        return x

In [9]:
class Args:
    def __init__(self, batch_size=32, test_batch_size=1, epochs=160, lr=0.0001, cuda=1, threads=16, seed=123,large_kernel_size = 9,small_kernel_size = 3,n_channels = 32,n_blocks = 16):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.cuda = cuda
        self.threads = threads
        self.seed = seed
        self.large_kernel_size =  large_kernel_size  # 第一层卷积和最后一层卷积的核大小
        self.small_kernel_size = small_kernel_size   # 中间层卷积的核大小
        self.n_channels = n_channels        # 中间层通道数
        self.n_blocks = n_blocks         # 残差模块数量
        #self.scaling_factor = scaling_factor
opt = Args()

print(opt)
print(opt.batch_size)
print(opt.n_channels)


32
32


In [10]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split

use_cuda = opt.cuda
if use_cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if use_cuda:
    torch.cuda.set_device(2)
    torch.cuda.manual_seed(opt.seed)
    

input_file_trainset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/input_only_2var_trainset.npy"
output_file_trainset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/mswxda_trainset_from_nc.npy"
input_file_testset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/input_only_2var_testset.npy"
output_file_testset = "/data/hanzhe/math_5410_program/temperature_downscaling/dataset/mswxda_testset_from_nc.npy"


train_dataset = LRHR_dataset(input_file_trainset,output_file_trainset)
test_dataset = LRHR_dataset(input_file_testset,output_file_testset)

train_mean_output,train_std_output = train_dataset.get_output_mean_std()
train_mean_input, train_std_input = train_dataset.get_input_i2_mean_std()
test_mean_output, test_std_output = test_dataset.get_output_mean_std()
test_mean_input, test_std_input = test_dataset.get_input_i2_mean_std()

training_data_loader = DataLoader(dataset=train_dataset, num_workers=opt.threads, batch_size=opt.batch_size, shuffle=True)
for batch_datas, batch_labels in training_data_loader:
    print(batch_datas.size(),batch_labels.size())
testing_data_loader = DataLoader(dataset=test_dataset, num_workers=opt.threads, batch_size=opt.test_batch_size, shuffle=False)

Variable 0 has at least one std=0, skipping normalization for this variable.
Variable 0 has at least one std=0, skipping normalization for this variable.
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.Size([32, 2, 64, 64]) torch.Size([32, 1, 64, 64])
torch.

In [11]:
from __future__ import print_function

from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torch
from torch import nn
import torchvision
import math




srcnn =  ViT(
    image_size=64, 
    patch_size=8, 
    num_classes=4096,  
    dim=512, 
    depth=6, 
    heads=8, 
    mlp_dim=2048, 
    channels=2,  
    pool='cls',  
    dim_head=64, 
)



criterion = nn.MSELoss()


if(use_cuda):
    srcnn.cuda()
    criterion = criterion.cuda()

optimizer = optim.Adam(srcnn.parameters(),lr=opt.lr)
result_path = '/data/hanzhe/math_5410_program/temperature_downscaling/vit/results'

def train(epoch):
    epoch_loss = 0
    srcnn.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = Variable(batch[0]), Variable(batch[1])
        if use_cuda:
            input = input.cuda()
            target = target.cuda()

        optimizer.zero_grad()
        #print ("input shape = " , input.shape)
        #print ("target shape = ", target.shape)
        model_out = srcnn(input)
        #print ("model_out shape =" , model_out.shape)
        loss = criterion(model_out, target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        writer.add_scalar('Loss/train', float(loss), epoch)

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))




def test(epoch):
    #avg_psnr = 0
    srcnn.eval()
    val_loss = 0
    avg_psnr = 0
    avg_mse = 0
    current_step = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            current_step +=1
            input, target = Variable(batch[0]), Variable(batch[1])
            if use_cuda:
                input = input.cuda()
                target = target.cuda()

            prediction = srcnn(input)
            loss = criterion(prediction, target)
            val_loss += loss.item()
            writer.add_scalar('Loss/test', float(loss), epoch)
            
            
            sr_image = prediction.detach().cpu()
            hr_image = target.detach().cpu()
            lr_image = input.detach().cpu()
            #print("sr shape:",sr_image.shape)
            #print("hr shape:",hr_image.shape)
            #print("lr shape:",lr_image.shape)
            
            
            sr_img = tensor2numpy(sr_image,test_mean_output,test_std_output,test_mean_input,test_std_input) # uint8
            hr_img = tensor2numpy(hr_image,test_mean_output,test_std_output,test_mean_input,test_std_input)  # uint8
            lr_img = tensor2numpy(lr_image,test_mean_output,test_std_output,test_mean_input,test_std_input) # uint8
            
            epoch_path = os.path.join(result_path, f'epoch_{epoch}')
            os.makedirs(epoch_path, exist_ok=True)
            
            # 保存图像
            save_img(hr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_hr.npy'))
            save_img(sr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_sr.npy'))
            save_img(lr_img, os.path.join(epoch_path, f'{epoch}_{current_step}_lr.npy'))
            
            
            psnr = calculate_psnr(sr_img, hr_img)
            mse = calculate_mse(sr_img, hr_img)
            #print(psnr)
            #print(calculate_mse(sr_img, hr_img))
            avg_psnr += psnr
            avg_mse+=mse
        print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))
        print("===> Avg. MSE: {:.4f} dB".format(avg_mse / len(testing_data_loader)))
    print("===> Epoch {} Complete test: Avg. Loss: {:.4f}".format(epoch, val_loss / len(testing_data_loader)))


def checkpoint(epoch):
    model_out_path = "model_epoch_{}.pth".format(epoch)
    torch.save(srcnn, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

for epoch in range(1, opt.epochs + 1):
    train(epoch)
    test(epoch)
    if(epoch%160==0):
        checkpoint(epoch)

===> Epoch[1](1/309): Loss: 1.1937
===> Epoch[1](2/309): Loss: 1.0500
===> Epoch[1](3/309): Loss: 1.1338
===> Epoch[1](4/309): Loss: 1.0774
===> Epoch[1](5/309): Loss: 0.8177
===> Epoch[1](6/309): Loss: 0.8521
===> Epoch[1](7/309): Loss: 0.7891
===> Epoch[1](8/309): Loss: 0.8669
===> Epoch[1](9/309): Loss: 0.9891
===> Epoch[1](10/309): Loss: 0.6084
===> Epoch[1](11/309): Loss: 1.1309
===> Epoch[1](12/309): Loss: 0.6974
===> Epoch[1](13/309): Loss: 1.2069
===> Epoch[1](14/309): Loss: 1.0964
===> Epoch[1](15/309): Loss: 0.7183
===> Epoch[1](16/309): Loss: 0.7418
===> Epoch[1](17/309): Loss: 0.5797
===> Epoch[1](18/309): Loss: 1.6626
===> Epoch[1](19/309): Loss: 1.3406
===> Epoch[1](20/309): Loss: 1.0238
===> Epoch[1](21/309): Loss: 0.7970
===> Epoch[1](22/309): Loss: 0.8961
===> Epoch[1](23/309): Loss: 0.6520
===> Epoch[1](24/309): Loss: 0.5132
===> Epoch[1](25/309): Loss: 0.7567
===> Epoch[1](26/309): Loss: 0.4778
===> Epoch[1](27/309): Loss: 0.6425
===> Epoch[1](28/309): Loss: 0.5661
=

===> Epoch 154 Complete: Avg. Loss: 0.0033
===> Avg. PSNR: 34.0380 dB
===> Avg. MSE: 0.0336 dB